In [ ]:
import win32com.client
import win32com.client.combrowse as com
from pyfmi import load_fmu
from PIL import Image
import matplotlib.pyplot as P
import numpy as np
import pandas as pd
import sqlite3
image1 = P.imread('ActiveLowPassFilter.png')
P.figure(figsize=(6,6))
P.axis(False)
P.imshow(image1)
image2 = P.imread('activelowpassSecondOrder.png')
P.figure(figsize=(6,6))
P.axis(False) 
P.imshow(image2)
image3 = P.imread('activehighpass.png')
P.figure(figsize=(6,6)) 
P.axis(False)
P.imshow(image3)
image4 = P.imread('activehighpasssecond.png')
P.figure(figsize=(6,6))
P.axis(False)
P.imshow(image4)
image5 = P.imread('multiplefeedback.png')
P.figure(figsize=(6,6))
P.axis(False)
P.imshow(image5)
image6 = P.imread('Bandstop.png')
P.figure(figsize=(6,6))
P.axis(False)
P.imshow(image6)
image7 = P.imread('rlcseries.png')
P.figure(figsize=(6,6))
P.axis(False)
P.imshow(image7)
image8 = P.imread('rlcparallel.png')
P.figure(figsize=(6,6))
P.axis(False)
P.imshow(image8)

In [ ]:
def lowfirstorder():
    model=load_fmu("activelowpass.fmu")
    objTKSolver = win32com.client.Dispatch("TKWX.Document")
    objTKSolver.LoadModel("r", r"C:\Users\HARDIK\UTS_PROJECT\ActiveLowPassFilter.tkwx")
    
    r1=input("Enter value of R1 ")
    r2=input("Enter value of R2 ")
    r3=input("Enter value of R3 ")
    c1=input("Enter value of C1 ")
    vin=input("Enter value of Voltage ")
    f=input("Enter value of frequency ")
    
    print("\n  SIMULATING....")
    model.set("R1.R",r1)
    model.set("R2.R",r2)
    model.set("R3.R",r3)
    model.set("C1.C",c1)
    model.set("sineVoltage.V",vin)
    model.set("sineVoltage.freqHz",f)
    
    res=model.simulate()
    
    objTKSolver.SetValue("R1", "i",r1)
    objTKSolver.SetValue("C", "i",c1)
    objTKSolver.SetValue("R2", "i",r2)
    objTKSolver.SetValue("R3", "i",r3)
    objTKSolver.SetValue("f", "i",f)
    objTKSolver.SetValue("vin", "i",vin)
    objTKSolver.SetValue("vout", "i",max(res['voltageSensor.v']))
    objTKSolver.Solve()
    fc=objTKSolver.GetValue("Fc", "o")
    Amax=objTKSolver.GetValue("Amax", "o")
    VoltageGain=objTKSolver.GetValue("VoltageGain", "o")
    voltage_gain=objTKSolver.GetValue("voltagegain", "o")
    gain_in_dB=objTKSolver.GetValue("gain_in_dB", "o")
    print("\n\n")
    print(" NOTE:-\n WHEN F<FC , Vout / Vin = Amax\n WHEN F=FC , Vout / Vin = 0.707 X Amax \n WHEN F>FC , Vout / Vin < Amax")
    print("\n")
    print("OUTPUT")
    print("Cut Off Frequency = ",fc,"Hz")
    print("Gain of the pass band = ",Amax)
    print("Voltage Gain (Amax / √{1+(f/fc)²) = ",VoltageGain)
    print("Voltage Gain (V_output/V_input) (from modelica) = ",voltage_gain)
    print("The gain in dB is given is 20log (Voltage Gain) =",gain_in_dB,"dB")
    print("\n\n")


    P.figure(figsize=(8,6))
    P.grid(True)
    P.plot(res['sineVoltage.v'],'r',linewidth=1.0,color="mediumvioletred",label='INPUT Voltage')
    P.plot(res['voltageSensor.v'],'r',linewidth=1.0,color="aqua",label='OUTPUT Voltage')
    P.title("INPUT OUPUT Voltages")
    P.ylabel('Voltage (V)')
    P.xlabel('Time')
    P.legend()
    P.show()

    x1=[]
    x11=[]
    y1=[]
    y11=[]
    z1=[]
    z11=[]
    
    
    f1=1
    f2=int(float(fc))*10
    for i in range(f1,f2,1):
        objTKSolver.SetSubCell("l",1,i,1,str(i))
    objTKSolver.ListSolve()
    index = objTKSolver.GetObjIndex("p","P1")
    prop1 = objTKSolver.GetProperty("p", index, 1)
    xlist = objTKSolver.GetSubCell("p", index, 1, 1)
    ylist = objTKSolver.GetSubCell("p", index, 1, 2)
    zlist = objTKSolver.GetSubCell("p", index, 2, 1)
    xListIndex = objTKSolver.GetObjIndex("l", xlist)
    yListIndex = objTKSolver.GetObjIndex("l", ylist)
    zListIndex = objTKSolver.GetObjIndex("l",zlist)
    for i in range(1,f2):
        x1.append(objTKSolver.GetSubCell("l", xListIndex, i, 1))
        x11.append(float(x1[i-1]))
    for i in  range(1,f2):
        y1.append(objTKSolver.GetSubCell("l", yListIndex, i, 1)) 
        y11.append(float(y1[i-1]))
    
    for i in  range(1,f2):
        z1.append(objTKSolver.GetSubCell("l", zListIndex, i, 1)) 
        z11.append(float(z1[i-1]))

    
    
    P.figure(figsize=(8,6))
    P.grid(True)
    P.xscale('log')
    
    P.plot(z11,y11,linewidth=1.5,color="limegreen",linestyle='dashed',label='Cut off frequency')
    P.plot(x11,y11,linewidth=2.0,color="magenta") 
    
    P. fill_between(x11, y11, color='lavender')
    P.title("VOLTAGE GAIN IN dB VS FREQUENCY")
    P.ylabel('Voltage Gain in dB')
    P.xlabel('Frequency (Hz)')
    P.legend()
    P.show()
    
    
    conn = sqlite3.connect('mydb.db')
    c = conn.cursor()
    c.execute("INSERT INTO Activelowpass1 (r1,r2,r3,c1,vin,f,fc,Amax,VoltageGain,voltage_gain,gain_in_dB) VALUES (?, ?, ?, ?,?, ?, ?, ?,?,?,?)",
                  (r1,r2,r3,c1,vin,f,fc,Amax,VoltageGain,voltage_gain,gain_in_dB))

    conn.commit()
    conn.close()
    



In [ ]:
def lowsecondorder():
    model = load_fmu ("activelowpassSecondOrder.fmu")
    objTKSolver = win32com.client.Dispatch("TKWX.Document")
    objTKSolver.LoadModel("r", r"C:\Users\HARDIK\UTS_PROJECT\activelowpassSecondOrder.tkwx")
    print("\n\n")
    R1=input("Enter value of Resistor 1")
    R2=input("Enter value of Resistor 2")
    R3=input("Enter value of Resistor 3")
    R4=input("Enter value of Resistor 4")
    C1=input("Enter value of Capacitor 1")
    C2=input("Enter value of Capacitor 2")
    f=input("Enter value of frequency")
    vin=input("Enter value of Sine Voltage")
    
    model.set("R1.R",R1)
    model.set("R2.R",R2)
    model.set("R3.R",R3)
    model.set("R4.R",R4)
    model.set("C1.C",C1)
    model.set("C2.C",C2)
    model.set("sineVoltage.V",vin)
    model.set("sineVoltage.freqHz",f)
    print("\n  SIMULATING....")
    res=model.simulate()
    objTKSolver.SetValue("R1", "i",R1)
    objTKSolver.SetValue("R2", "i",R2)
    objTKSolver.SetValue("R3", "i",R3)
    objTKSolver.SetValue("R4", "i",R4)
    objTKSolver.SetValue("C1", "i",C1)
    objTKSolver.SetValue("C2", "i",C2)
    objTKSolver.SetValue("f", "i",f)
    objTKSolver.SetValue("vin", "i",vin)
    objTKSolver.SetValue("vout", "i",max(res['voltageSensor.v']))
    objTKSolver.Solve()
    fc=objTKSolver.GetValue("Fc", "o")
    Av=objTKSolver.GetValue("Av", "o")
    voltage_gain=objTKSolver.GetValue("voltagegain", "o")
    gain_in_dB=objTKSolver.GetValue("gain_in_dB", "o")
    VoltageGain=objTKSolver.GetValue("VoltageGain", "o")
    print("\n\n")
    print(" NOTE:-\n WHEN F<FC , Vout / Vin = Amax\n WHEN F=FC , Vout / Vin = 0.707 X Amax \n WHEN F>FC , Vout / Vin < Amax")
    print("\n")
    print("OUTPUT")
    print("Voltage Output = ",max(res['voltageSensor.v']),"V")
    print("Cut Off Frequency = ",fc,"Hz")
    print("Gain of the pass band = ",Av)
    print("Voltage Gain (V_output/V_input) (from modelica) = ",voltage_gain)
    print("Voltage Gain (Av / √{1+(f/fc)^4) = ",VoltageGain)
    print("The gain in dB is given is 20log (Voltage Gain) =",gain_in_dB,"dB")
    print("\n\n")


    P.figure(figsize=(8,6))
    P.grid(True)
    P.plot(res['sineVoltage.v'],'r',linewidth=1.0,color="limegreen",label='INPUT Voltage')
    P.plot(res['voltageSensor.v'],'r',linewidth=1.0,color="orange",label='OUTPUT Voltage')
    P.title("INPUT OUPUT Voltages")
    P.ylabel('Voltage (V)')
    P.xlabel('Time')
    P.legend()
    P.show()


    x1=[]
    x11=[]
    y1=[]
    y11=[]
    z1=[]
    z11=[]
    
    
    f1=1
    f2=int(float(fc))*10
    for i in range(f1,f2,1):
        objTKSolver.SetSubCell("l",1,i,1,str(i))
    objTKSolver.ListSolve()
    index = objTKSolver.GetObjIndex("p","P1")
    prop1 = objTKSolver.GetProperty("p", index, 1)
    xlist = objTKSolver.GetSubCell("p", index, 1, 1)
    ylist = objTKSolver.GetSubCell("p", index, 1, 2)
    zlist = objTKSolver.GetSubCell("p", index, 2, 1)
    xListIndex = objTKSolver.GetObjIndex("l", xlist)
    yListIndex = objTKSolver.GetObjIndex("l", ylist)
    zListIndex = objTKSolver.GetObjIndex("l",zlist)
    for i in range(1,f2):
        x1.append(objTKSolver.GetSubCell("l", xListIndex, i, 1))
        x11.append(float(x1[i-1]))
    for i in  range(1,f2):
        y1.append(objTKSolver.GetSubCell("l", yListIndex, i, 1)) 
        y11.append(float(y1[i-1]))
    
    for i in  range(1,f2):
        z1.append(objTKSolver.GetSubCell("l", zListIndex, i, 1)) 
        z11.append(float(z1[i-1]))

    
    
    P.figure(figsize=(8,6))
    P.grid(True)
    P.xscale('log')
    P.plot(z11,y11,linewidth=1.5,color="limegreen",linestyle='dashed',label='Cut off frequency')
    P.plot(x11,y11,linewidth=2.0,color="magenta") 
    P. fill_between(x11, y11, color='lavender')
    P.title("VOLTAGE GAIN IN dB VS FREQUENCY")
    P.ylabel('Voltage Gain in dB')
    P.xlabel('Frequency (Hz)')
    P.legend()
    P.show()
    conn = sqlite3.connect('mydb.db')
    c = conn.cursor()
    c.execute("INSERT INTO Activelowpass2 (R1,R2,R3,R4,C1,C2,f,vin,fc,Av,voltage_gain,gain_in_dB,VoltageGain) VALUES (?, ?, ?, ?,?, ?, ?, ?,?,?,?,?,?)",
                  (R1,R2,R3,R4,C1,C2,f,vin,fc,Av,voltage_gain,gain_in_dB,VoltageGain))

    conn.commit()
    conn.close()
    

In [ ]:
def highpass_FirstAndSecondOrder():
    
    model1=load_fmu("activehighpass.fmu")
    objTKSolver1 = win32com.client.Dispatch("TKWX.Document")
    objTKSolver1.LoadModel("r", r"C:\Users\HARDIK\UTS_PROJECT\activehighpass.tkwx")
    
    model2= load_fmu ("activehighpasssecond.fmu")
    objTKSolver2= win32com.client.Dispatch("TKWX.Document")
    objTKSolver2.LoadModel("r", r"C:\Users\HARDIK\UTS_PROJECT\activehighpasssecond.tkwx")
    
    R1=input("Enter value of Resistor 1")
    R2=input("Enter value of Resistor 2")
    R3=input("Enter value of Resistor 3")
    R4=input("Enter value of Resistor 4")
    C1=input("Enter value of Capacitor 1")
    C2=input("Enter value of Capacitor 2")
    f=input("Enter value of frequency")
    vin=input("Enter value of Sine Voltage")
    
    print("\n  SIMULATING....\n")
    model1.set("r1.R",R1)
    model1.set("r2.R",R2)
    model1.set("r3.R",R3)
    model1.set("c1.C",C1)
    model1.set("sineVoltage.V",vin)
    model1.set("sineVoltage.freqHz",f)
    
    model2.set("resistor1.R",R1)
    model2.set("resistor2.R",R2)
    model2.set("r3.R",R3)
    model2.set("r4.R",R4)
    model2.set("c1.C",C1)
    model2.set("c2.C",C2)
    model2.set("sineVoltage.V",vin)
    model2.set("sineVoltage.freqHz",f)
    
    res1=model1.simulate()
    res2=model2.simulate()
    
    objTKSolver1.SetValue("R1", "i",R1)
    objTKSolver1.SetValue("C", "i",C1)
    objTKSolver1.SetValue("R2", "i",R2)
    objTKSolver1.SetValue("R3", "i",R3)
    objTKSolver1.SetValue("f", "i",f)
    objTKSolver1.SetValue("vin", "i",vin)
    objTKSolver1.SetValue("vout", "i",max(res1['opAmp.out.v']))
    objTKSolver1.Solve()
    fc1=objTKSolver1.GetValue("Fc", "o")
    Amax1=objTKSolver1.GetValue("Amax", "o")
    VoltageGain1=objTKSolver1.GetValue("VoltageGain", "o")
    voltage_gain1=objTKSolver1.GetValue("voltagegain", "o")
    gain_in_dB1=objTKSolver1.GetValue("gain_in_dB", "o")
    print("\n\n")
    print("FIRST ORDER OUTPUT :-")
    print("\n")
    print("Cut Off Frequency = ",fc1,"Hz")
    print("Gain of the pass band = ",Amax1)
    print("Voltage Gain (Amax / √{1+(f/fc)²) = ",VoltageGain1)
    print("Voltage Gain (V_output/V_input) (from modelica) = ",voltage_gain1)
    print("The gain in dB is given is 20log (Voltage Gain) =",gain_in_dB1,"dB")
    print("\n\n")

    
    
    objTKSolver2.SetValue("R1", "i",R1)
    objTKSolver2.SetValue("R2", "i",R2)
    objTKSolver2.SetValue("R3", "i",R3)
    objTKSolver2.SetValue("R4", "i",R4)
    objTKSolver2.SetValue("C1", "i",C1)
    objTKSolver2.SetValue("C2", "i",C2)
    objTKSolver2.SetValue("f", "i",f)
    objTKSolver2.SetValue("vin", "i",vin)
    objTKSolver2.SetValue("vout", "i",max(res2['opAmp.out.v']))
    objTKSolver2.Solve()
    fc2=objTKSolver2.GetValue("Fc", "o")
    Av2=objTKSolver2.GetValue("Av", "o")
    voltage_gain2=objTKSolver2.GetValue("voltagegain", "o")
    gain_in_dB2=objTKSolver2.GetValue("gain_in_dB", "o")
    VoltageGain2=objTKSolver2.GetValue("VoltageGain", "o")
   
    print("SECOND ORDER OUTPUT:- ")
    print("\n")
    print("Voltage Output = ",max(res2['opAmp.out.v']),"V")
    print("Cut Off Frequency = ",fc2,"Hz")
    print("Gain of the pass band = ",Av2)
    print("Voltage Gain (V_output/V_input) (from modelica) = ",voltage_gain2)
    print("Voltage Gain (Av / √{1+(f/fc)^4) = ",VoltageGain2)
    print("The gain in dB is given is 20log (Voltage Gain) =",gain_in_dB2,"dB")
    print("\n\n")

    P.figure(figsize=(10,8))
    P.grid(True)
    P.plot(res1['sineVoltage.v'],'r',linewidth=1.0,color="mediumvioletred",label='INPUT Voltage')
    P.plot(res1['opAmp.out.v'],'r',linewidth=1.0,color="aqua",label='OUTPUT Voltage (FIRST ORDER)')
    P.plot(res2['opAmp.out.v'],'r',linewidth=1.0,color="limegreen",label='OUTPUT Voltage (SECOND ORDER)')
    P.title("INPUT OUPUT Voltages")
    P.ylabel('Voltage (V)')
    P.xlabel('Time')
    P.yscale("log")
    P.legend()
    P.show()

    x1_1=[]
    x11_1=[]
    y1_1=[]
    y11_1=[]
    z1_1=[]
    z11_1=[]
    
    
    f1_1=1
    f2_1=int(float(fc1))*10
    if(f2_1>64000):
        f2_1=64000
    for i in range(f1_1,f2_1,1):
        objTKSolver1.SetSubCell("l",1,i,1,str(i))
    objTKSolver1.ListSolve()
    index_1 = objTKSolver1.GetObjIndex("p","P1")
    prop1_1 = objTKSolver1.GetProperty("p", index_1, 1)
    xlist_1 = objTKSolver1.GetSubCell("p", index_1, 1, 1)
    ylist_1 = objTKSolver1.GetSubCell("p", index_1, 1, 2)
    zlist_1 = objTKSolver1.GetSubCell("p", index_1, 2, 1)
    xListIndex_1 = objTKSolver1.GetObjIndex("l", xlist_1)
    yListIndex_1 = objTKSolver1.GetObjIndex("l", ylist_1)
    zListIndex_1 = objTKSolver1.GetObjIndex("l",zlist_1)
    for i in range(1,f2_1):
        x1_1.append(objTKSolver1.GetSubCell("l", xListIndex_1, i, 1))
        x11_1.append(float(x1_1[i-1]))
    for i in  range(1,f2_1):
        y1_1.append(objTKSolver1.GetSubCell("l", yListIndex_1, i, 1)) 
        y11_1.append(float(y1_1[i-1]))
    
    for i in  range(1,f2_1):
        z1_1.append(objTKSolver1.GetSubCell("l", zListIndex_1, i, 1)) 
        z11_1.append(float(z1_1[i-1]))

    
    
    x1_2=[]
    x11_2=[]
    y1_2=[]
    y11_2=[]
    z1_2=[]
    z11_2=[]
    
    
    f1_2=1
    f2_2=int(float(fc2))*10
    if(f2_2>64000):
        f2_2=64000
    for i in range(f1_2,f2_2,1):
        objTKSolver2.SetSubCell("l",1,i,1,str(i))
    objTKSolver2.ListSolve()
    index_2 = objTKSolver2.GetObjIndex("p","P1")
    prop1_2 = objTKSolver2.GetProperty("p", index_2, 1)
    xlist_2 = objTKSolver2.GetSubCell("p", index_2, 1, 1)
    ylist_2 = objTKSolver2.GetSubCell("p", index_2, 1, 2)
    zlist_2 = objTKSolver2.GetSubCell("p", index_2, 2, 1)
    xListIndex_2 = objTKSolver2.GetObjIndex("l", xlist_2)
    yListIndex_2 = objTKSolver2.GetObjIndex("l", ylist_2)
    zListIndex_2 = objTKSolver2.GetObjIndex("l",zlist_2)
    for i in range(1,f2_2):
        x1_2.append(objTKSolver2.GetSubCell("l", xListIndex_2, i, 1))
        x11_2.append(float(x1_2[i-1]))
    for i in  range(1,f2_2):
        y1_2.append(objTKSolver2.GetSubCell("l", yListIndex_2, i, 1)) 
        y11_2.append(float(y1_2[i-1]))
    
    for i in  range(1,f2_2):
        z1_2.append(objTKSolver2.GetSubCell("l", zListIndex_2, i, 1)) 
        z11_2.append(float(z1_2[i-1]))

    
    
    P.figure(figsize=(10,8))
    P.grid(True)
    P.xscale('log')
    P.plot(z11_1,y11_1,linewidth=1.5,color="orange",linestyle='dashed',label='Cut off frequency (FIRST ORDER)')
    P.plot(x11_1,y11_1,linewidth=2.0,color="magenta",label="Voltage Gain in dB (FIRST ORDER)") 
    P.plot(z11_2,y11_2,linewidth=1.5,color="blue",linestyle='dashed',label='Cut off frequency (SECOND ORDER)')
    P.plot(x11_2,y11_2,linewidth=2.0,color="limegreen",label="Voltage Gain in dB (SECOND ORDER)") 
    #P.fill_between(x11_1, y11_1, color='lavenderblush')
    #P.fill_between(x11_2, y11_2, color='lavenderblush')
    P.title("VOLTAGE GAIN IN dB VS FREQUENCY")
    P.ylabel('Voltage Gain in dB')
    P.xlabel('Frequency (Hz)')
    P.legend()
    P.show()
    conn = sqlite3.connect('mydb.db')
    c = conn.cursor()
    c.execute("INSERT INTO Activehighpass (R1,R2,R3,R4,C1,C2,f,vin,fc1,Amax1,voltage_gain1,gain_in_dB1,VoltageGain1,fc2,Av2,voltage_gain2,gain_in_dB2,VoltageGain2) VALUES (?, ?, ?, ?,?, ?, ?, ?,?,?,?,?,?,?,?,?,?,?)",
                  (R1,R2,R3,R4,C1,C2,f,vin,fc1,Amax1,voltage_gain1,gain_in_dB1,VoltageGain1,fc2,Av2,voltage_gain2,gain_in_dB2,VoltageGain2))

    conn.commit()
    conn.close()

    

In [ ]:
def lowpass_FirstAndSecondOrder():
    
    model1=load_fmu("activelowpass.fmu")
    objTKSolver1 = win32com.client.Dispatch("TKWX.Document")
    objTKSolver1.LoadModel("r", r"C:\Users\HARDIK\UTS_PROJECT\ActiveLowPassFilter.tkwx")
    
    model2= load_fmu ("activelowpassSecondOrder.fmu")
    objTKSolver2= win32com.client.Dispatch("TKWX.Document")
    objTKSolver2.LoadModel("r", r"C:\Users\HARDIK\UTS_PROJECT\activelowpassSecondOrder.tkwx")
    
    R1=input("Enter value of Resistor 1")
    R2=input("Enter value of Resistor 2")
    R3=input("Enter value of Resistor 3")
    R4=input("Enter value of Resistor 4")
    C1=input("Enter value of Capacitor 1")
    C2=input("Enter value of Capacitor 2")
    f=input("Enter value of frequency")
    vin=input("Enter value of Sine Voltage")
    
    print("\n  SIMULATING....\n")
    model1.set("R1.R",R1)
    model1.set("R2.R",R2)
    model1.set("R3.R",R3)
    model1.set("C1.C",C1)
    model1.set("sineVoltage.V",vin)
    model1.set("sineVoltage.freqHz",f)
    
    model2.set("R1.R",R1)
    model2.set("R2.R",R2)
    model2.set("R3.R",R3)
    model2.set("R4.R",R4)
    model2.set("C1.C",C1)
    model2.set("C2.C",C2)
    model2.set("sineVoltage.V",vin)
    model2.set("sineVoltage.freqHz",f)
    
    res1=model1.simulate()
    res2=model2.simulate()
    
    objTKSolver1.SetValue("R1", "i",R1)
    objTKSolver1.SetValue("C", "i",C1)
    objTKSolver1.SetValue("R2", "i",R2)
    objTKSolver1.SetValue("R3", "i",R3)
    objTKSolver1.SetValue("f", "i",f)
    objTKSolver1.SetValue("vin", "i",vin)
    objTKSolver1.SetValue("vout", "i",max(res1['voltageSensor.v']))
    objTKSolver1.Solve()
    fc1=objTKSolver1.GetValue("Fc", "o")
    Amax1=objTKSolver1.GetValue("Amax", "o")
    VoltageGain1=objTKSolver1.GetValue("VoltageGain", "o")
    voltage_gain1=objTKSolver1.GetValue("voltagegain", "o")
    gain_in_dB1=objTKSolver1.GetValue("gain_in_dB", "o")
    print("\n\n")
    print("FIRST ORDER OUTPUT :-")
    print("\n")
    print("Cut Off Frequency = ",fc1,"Hz")
    print("Gain of the pass band = ",Amax1)
    print("Voltage Gain (Amax / √{1+(f/fc)²) = ",VoltageGain1)
    print("Voltage Gain (V_output/V_input) (from modelica) = ",voltage_gain1)
    print("The gain in dB is given is 20log (Voltage Gain) =",gain_in_dB1,"dB")
    print("\n\n")

    
    
    objTKSolver2.SetValue("R1", "i",R1)
    objTKSolver2.SetValue("R2", "i",R2)
    objTKSolver2.SetValue("R3", "i",R3)
    objTKSolver2.SetValue("R4", "i",R4)
    objTKSolver2.SetValue("C1", "i",C1)
    objTKSolver2.SetValue("C2", "i",C2)
    objTKSolver2.SetValue("f", "i",f)
    objTKSolver2.SetValue("vin", "i",vin)
    objTKSolver2.SetValue("vout", "i",max(res2['voltageSensor.v']))
    objTKSolver2.Solve()
    fc2=objTKSolver2.GetValue("Fc", "o")
    Av2=objTKSolver2.GetValue("Av", "o")
    voltage_gain2=objTKSolver2.GetValue("voltagegain", "o")
    gain_in_dB2=objTKSolver2.GetValue("gain_in_dB", "o")
    VoltageGain2=objTKSolver2.GetValue("VoltageGain", "o")
   
    print("SECOND ORDER OUTPUT:- ")
    print("\n")
    print("Voltage Output = ",max(res2['voltageSensor.v']),"V")
    print("Cut Off Frequency = ",fc2,"Hz")
    print("Gain of the pass band = ",Av2)
    print("Voltage Gain (V_output/V_input) (from modelica) = ",voltage_gain2)
    print("Voltage Gain (Av / √{1+(f/fc)^4) = ",VoltageGain2)
    print("The gain in dB is given is 20log (Voltage Gain) =",gain_in_dB2,"dB")
    print("\n\n")

    P.figure(figsize=(10,8))
    P.grid(True)
    P.plot(res1['sineVoltage.v'],'r',linewidth=1.0,color="mediumvioletred",label='INPUT Voltage')
    P.plot(res1['voltageSensor.v'],'r',linewidth=1.0,color="limegreen",label='OUTPUT Voltage (FIRST ORDER)')
    P.plot(res2['voltageSensor.v'],'r',linewidth=1.0,color="orange",label='OUTPUT Voltage (SECOND ORDER)')
    P.title("INPUT OUPUT Voltages")
    P.ylabel('Voltage (V)')
    P.xlabel('Time')
    P.legend()
    P.show()

    x1_1=[]
    x11_1=[]
    y1_1=[]
    y11_1=[]
    z1_1=[]
    z11_1=[]
    
    
    f1_1=1
    f2_1=int(float(fc1))*10
    for i in range(f1_1,f2_1,1):
        objTKSolver1.SetSubCell("l",1,i,1,str(i))
    objTKSolver1.ListSolve()
    index_1 = objTKSolver1.GetObjIndex("p","P1")
    prop1_1 = objTKSolver1.GetProperty("p", index_1, 1)
    xlist_1 = objTKSolver1.GetSubCell("p", index_1, 1, 1)
    ylist_1 = objTKSolver1.GetSubCell("p", index_1, 1, 2)
    zlist_1 = objTKSolver1.GetSubCell("p", index_1, 2, 1)
    xListIndex_1 = objTKSolver1.GetObjIndex("l", xlist_1)
    yListIndex_1 = objTKSolver1.GetObjIndex("l", ylist_1)
    zListIndex_1 = objTKSolver1.GetObjIndex("l",zlist_1)
    for i in range(1,f2_1):
        x1_1.append(objTKSolver1.GetSubCell("l", xListIndex_1, i, 1))
        x11_1.append(float(x1_1[i-1]))
    for i in  range(1,f2_1):
        y1_1.append(objTKSolver1.GetSubCell("l", yListIndex_1, i, 1)) 
        y11_1.append(float(y1_1[i-1]))
    
    for i in  range(1,f2_1):
        z1_1.append(objTKSolver1.GetSubCell("l", zListIndex_1, i, 1)) 
        z11_1.append(float(z1_1[i-1]))

    
    
    x1_2=[]
    x11_2=[]
    y1_2=[]
    y11_2=[]
    z1_2=[]
    z11_2=[]
    
    
    f1_2=1
    f2_2=int(float(fc2))*10
    for i in range(f1_2,f2_2,1):
        objTKSolver2.SetSubCell("l",1,i,1,str(i))
    objTKSolver2.ListSolve()
    index_2 = objTKSolver2.GetObjIndex("p","P1")
    prop1_2 = objTKSolver2.GetProperty("p", index_2, 1)
    xlist_2 = objTKSolver2.GetSubCell("p", index_2, 1, 1)
    ylist_2 = objTKSolver2.GetSubCell("p", index_2, 1, 2)
    zlist_2 = objTKSolver2.GetSubCell("p", index_2, 2, 1)
    xListIndex_2 = objTKSolver2.GetObjIndex("l", xlist_2)
    yListIndex_2 = objTKSolver2.GetObjIndex("l", ylist_2)
    zListIndex_2 = objTKSolver2.GetObjIndex("l",zlist_2)
    for i in range(1,f2_2):
        x1_2.append(objTKSolver2.GetSubCell("l", xListIndex_2, i, 1))
        x11_2.append(float(x1_2[i-1]))
    for i in  range(1,f2_2):
        y1_2.append(objTKSolver2.GetSubCell("l", yListIndex_2, i, 1)) 
        y11_2.append(float(y1_2[i-1]))
    
    for i in  range(1,f2_2):
        z1_2.append(objTKSolver2.GetSubCell("l", zListIndex_2, i, 1)) 
        z11_2.append(float(z1_2[i-1]))

    
    
    P.figure(figsize=(10,8))
    P.grid(True)
    P.xscale('log')
    P.plot(z11_1,y11_1,linewidth=1.5,color="orange",linestyle='dashed',label='Cut off frequency (FIRST ORDER)')
    P.plot(x11_1,y11_1,linewidth=2.0,color="magenta",label="Voltage Gain in dB (FIRST ORDER)") 
    P.plot(z11_2,y11_2,linewidth=1.5,color="blue",linestyle='dashed',label='Cut off frequency (SECOND ORDER)')
    P.plot(x11_2,y11_2,linewidth=2.0,color="limegreen",label="Voltage Gain in dB (SECOND ORDER)") 
    P.fill_between(x11_1, y11_1, color='lavenderblush')
    P.fill_between(x11_2, y11_2, color='lavenderblush')
    P.title("VOLTAGE GAIN IN dB VS FREQUENCY")
    P.ylabel('Voltage Gain in dB')
    P.xlabel('Frequency (Hz)')
    P.legend()
    P.show()
    conn = sqlite3.connect('mydb.db')
    c = conn.cursor()
    c.execute("INSERT INTO Activelowpass (R1,R2,R3,R4,C1,C2,f,vin,fc1,Amax1,voltage_gain1,gain_in_dB1,VoltageGain1,fc2,Av2,voltage_gain2,gain_in_dB2,VoltageGain2) VALUES (?, ?, ?, ?,?, ?, ?, ?,?,?,?,?,?,?,?,?,?,?)",
                  (R1,R2,R3,R4,C1,C2,f,vin,fc1,Amax1,voltage_gain1,gain_in_dB1,VoltageGain1,fc2,Av2,voltage_gain2,gain_in_dB2,VoltageGain2))

    conn.commit()
    conn.close()
   

In [ ]:
def highfirstorder():
    
    model=load_fmu("activehighpass.fmu")
    objTKSolver = win32com.client.Dispatch("TKWX.Document")
    objTKSolver.LoadModel("r", r"C:\Users\HARDIK\UTS_PROJECT\activehighpass.tkwx")
    
    r1=input("Enter value of R1 ")
    r2=input("Enter value of R2 ")
    r3=input("Enter value of R3 ")
    c1=input("Enter value of C1 ")
    vin=input("Enter value of Voltage ")
    f=input("Enter value of frequency ")
    print("\n  SIMULATING....")
    model.set("r1.R",r1)
    model.set("r2.R",r2)
    model.set("r3.R",r3)
    model.set("c1.C",c1)
    model.set("sineVoltage.V",vin)
    model.set("sineVoltage.freqHz",f)
    res=model.simulate()
    objTKSolver.SetValue("R1", "i",r1)
    objTKSolver.SetValue("C", "i",c1)
    objTKSolver.SetValue("R2", "i",r2)
    objTKSolver.SetValue("R3", "i",r3)
    objTKSolver.SetValue("f", "i",f)
    objTKSolver.SetValue("vin", "i",vin)
    objTKSolver.SetValue("vout", "i",max(res['opAmp.out.v']))
    objTKSolver.Solve()
    fc=objTKSolver.GetValue("Fc", "o")
    Amax=objTKSolver.GetValue("Amax", "o")
    VoltageGain=objTKSolver.GetValue("VoltageGain", "o")
    voltage_gain=objTKSolver.GetValue("voltagegain", "o")
    gain_in_dB=objTKSolver.GetValue("gain_in_dB", "o")
    phi=objTKSolver.GetValue("phi", "o")
    print("\n\n")
    
    print("\n")
    print("OUTPUT")
    print("Cut Off Frequency = ",fc,"Hz")
    print("Gain of the pass band = ",Amax)
    print("Voltage Gain (Amax*(f/fc) / √{1+(f/fc)²) = ",VoltageGain)
    print("Voltage Gain (V_output/V_input) (from modelica)= ",voltage_gain)
    print("The gain in dB is given is 20log (Voltage Gain) =",gain_in_dB,"dB")
    print("Phase shift = ",phi)
    print("\n\n")


    P.figure(figsize=(8,6))
    P.grid(True)
    P.plot(res['opAmp.out.v'],'r',linewidth=1.0,color="aqua",label='OUTPUT Voltage')
    P.title("OUPUT Voltage")
    P.ylabel('Voltage (V)')
    P.xlabel('Time')
    P.xscale("log")
    P.legend()
    P.show()
    
    x1=[]
    x11=[]
    y1=[]
    y11=[]
    z1=[]
    z11=[]
    
    
    f1=1
    f2=int(float(fc))*10
    if(f2>64000):
        f2=64000
    for i in range(f1,f2,1):
        objTKSolver.SetSubCell("l",1,i,1,str(i))
    objTKSolver.ListSolve()
    index = objTKSolver.GetObjIndex("p","P1")
    prop1 = objTKSolver.GetProperty("p", index, 1)
    xlist = objTKSolver.GetSubCell("p", index, 1, 1)
    ylist = objTKSolver.GetSubCell("p", index, 1, 2)
    zlist = objTKSolver.GetSubCell("p", index, 2, 1)
    xListIndex = objTKSolver.GetObjIndex("l", xlist)
    yListIndex = objTKSolver.GetObjIndex("l", ylist)
    zListIndex = objTKSolver.GetObjIndex("l",zlist)
    for i in range(1,f2):
        x1.append(objTKSolver.GetSubCell("l", xListIndex, i, 1))
        x11.append(float(x1[i-1]))
    for i in  range(1,f2):
        y1.append(objTKSolver.GetSubCell("l", yListIndex, i, 1)) 
        y11.append(float(y1[i-1]))
    
    for i in  range(1,f2):
        z1.append(objTKSolver.GetSubCell("l", zListIndex, i, 1)) 
        z11.append(float(z1[i-1]))

    
    
    P.figure(figsize=(8,6))
    P.grid(True)
    P.xscale('log')
    P.plot(z11,y11,linewidth=1.5,color="limegreen",linestyle='dashed',label='Cut off frequency')
    P.plot(x11,y11,linewidth=2.0,color="magenta") 
    #P. fill_between(x11, y11,np.max(y11),color='lavender')
    P.title("VOLTAGE GAIN IN dB VS FREQUENCY")
    P.ylabel('Voltage Gain in dB')
    P.xlabel('Frequency (Hz)')
    P.show()
    P.show()
    conn = sqlite3.connect('mydb.db')
    c = conn.cursor()
    c.execute("INSERT INTO Activehighpass1 (r1,r2,r3,c1,vin,f,fc,Amax,VoltageGain,voltage_gain,gain_in_dB) VALUES (?, ?, ?, ?,?, ?, ?, ?,?,?,?)",
            (r1, r2, r3, c1, vin, f, fc, Amax, VoltageGain, voltage_gain, gain_in_dB))

    conn.commit()
    conn.close()
    


    
    

In [ ]:
def highsecondorder():
    model = load_fmu ("activehighpasssecond.fmu")
    objTKSolver = win32com.client.Dispatch("TKWX.Document")
    objTKSolver.LoadModel("r", r"C:\Users\HARDIK\UTS_PROJECT\activehighpasssecond.tkwx")
    print("\n\n")
    R1=input("Enter value of Resistor 1")
    R2=input("Enter value of Resistor 2")
    R3=input("Enter value of Resistor 3")
    R4=input("Enter value of Resistor 4")
    C1=input("Enter value of Capacitor 1")
    C2=input("Enter value of Capacitor 2")
    f=input("Enter value of frequency")
    vin=input("Enter value of Sine Voltage")
    
    model.set("resistor1.R",R1)
    model.set("resistor2.R",R2)
    model.set("r3.R",R3)
    model.set("r4.R",R4)
    model.set("c1.C",C1)
    model.set("c2.C",C2)
    model.set("sineVoltage.V",vin)
    model.set("sineVoltage.freqHz",f)
    print("\n  SIMULATING....")
    res=model.simulate()
    objTKSolver.SetValue("R1", "i",R1)
    objTKSolver.SetValue("R2", "i",R2)
    objTKSolver.SetValue("R3", "i",R3)
    objTKSolver.SetValue("R4", "i",R4)
    objTKSolver.SetValue("C1", "i",C1)
    objTKSolver.SetValue("C2", "i",C2)
    objTKSolver.SetValue("f", "i",f)
    objTKSolver.SetValue("vin", "i",vin)
    objTKSolver.SetValue("vout", "i",max(res['opAmp.out.v']))
    objTKSolver.Solve()
    fc=objTKSolver.GetValue("Fc", "o")
    Av=objTKSolver.GetValue("Av", "o")
    voltage_gain=objTKSolver.GetValue("voltagegain", "o")
    gain_in_dB=objTKSolver.GetValue("gain_in_dB", "o")
    VoltageGain=objTKSolver.GetValue("VoltageGain", "o")
    print("\n\n")
    
    print("\n")
    print("OUTPUT")
            
    print("Voltage Output = ",max(res['opAmp.out.v']))
    print("Cut Off Frequency = ",fc,"Hz")
    print("Gain of the pass band = ",Av)
    print("Voltage Gain (V_output/V_input) (from modelica)= ",voltage_gain)
    print("Voltage Gain (Av / √{1+(f/fc)^4) = ",VoltageGain)
    print("The gain in dB is given is 20log (Voltage Gain) =",gain_in_dB,"dB")
    print("\n\n")
    
    P.figure(figsize=(8,6))
    P.grid(True)
    #P.plot(res['sineVoltage.v'],'r',linewidth=1.0,color="limegreen",label='INPUT Voltage')
    P.plot(res['opAmp.out.v'],'r',linewidth=1.0,color="orange",label='OUTPUT Voltage')
    P.title("OUPUT Voltage")
    P.ylabel('Voltage (V)')
    P.xlabel('Time')
    P.legend()
    P.show()

   
    x1=[]
    x11=[]
    y1=[]
    y11=[]
    z1=[]
    z11=[]
    
    
    f1=1
    f2=int(float(fc))*10
    if(f2>64000):
        f2=64000
    for i in range(f1,f2,1):
        objTKSolver.SetSubCell("l",1,i,1,str(i))
    objTKSolver.ListSolve()
    index = objTKSolver.GetObjIndex("p","P1")
    prop1 = objTKSolver.GetProperty("p", index, 1)
    xlist = objTKSolver.GetSubCell("p", index, 1, 1)
    ylist = objTKSolver.GetSubCell("p", index, 1, 2)
    zlist = objTKSolver.GetSubCell("p", index, 2, 1)
    xListIndex = objTKSolver.GetObjIndex("l", xlist)
    yListIndex = objTKSolver.GetObjIndex("l", ylist)
    zListIndex = objTKSolver.GetObjIndex("l",zlist)
    for i in range(1,f2):
        x1.append(objTKSolver.GetSubCell("l", xListIndex, i, 1))
        x11.append(float(x1[i-1]))
    for i in  range(1,f2):
        y1.append(objTKSolver.GetSubCell("l", yListIndex, i, 1)) 
        y11.append(float(y1[i-1]))
    
    for i in  range(1,f2):
        z1.append(objTKSolver.GetSubCell("l", zListIndex, i, 1)) 
        z11.append(float(z1[i-1]))

    
    
    P.figure(figsize=(8,6))
    P.grid(True)
    P.xscale('log')
    P.plot(z11,y11,linewidth=1.5,color="limegreen",linestyle='dashed',label='Cut off frequency')
    P.plot(x11,y11,linewidth=2.0,color="magenta") 
    #P. fill_between(x11, y11,np.max(x11),color='lavender')
    P.title("VOLTAGE GAIN IN dB VS FREQUENCY")
    P.ylabel('Voltage Gain in dB')
    P.xlabel('Frequency (Hz)')
    P.legend()
    P.show()
    conn = sqlite3.connect('mydb.db')
    c = conn.cursor()
    c.execute("INSERT INTO Activehighpass2 (R1,R2,R3,R4,C1,C2,f,vin,fc,Av,voltage_gain,gain_in_dB,VoltageGain) VALUES (?, ?, ?, ?,?, ?, ?, ?,?,?,?,?,?)",
                  (R1,R2,R3,R4,C1,C2,f,vin,fc,Av,voltage_gain,gain_in_dB,VoltageGain))

    conn.commit()
    conn.close()
    
    
    


   

In [ ]:
def multiplefeedback():
    objTKSolver = win32com.client.Dispatch("TKWX.Document")
    objTKSolver.LoadModel("r", r"C:\Users\HARDIK\UTS_PROJECT\multiplefeedback.tkwx")
    print("\n\n")
    R1=input("Enter value of Resistor 1")
    R2=input("Enter value of Resistor 2")
    R3=input("Enter value of Resistor 3")
    C1=input("Enter value of Capacitor 1")
    C2=input("Enter value of Capacitor 2")
    vin=input("Enter value of Sine Voltage")
    Fc1=input("Enter value of Fc1 ")
    Fc2=input("Enter value of Fc2 ")
    objTKSolver.SetValue("R1", "i",R1)
    objTKSolver.SetValue("R2", "i",R2)
    objTKSolver.SetValue("R3", "i",R3)
    objTKSolver.SetValue("C1", "i",C1)
    objTKSolver.SetValue("C2", "i",C2)
    objTKSolver.SetValue("vin", "i",vin)
    objTKSolver.SetValue("Fc1", "i",Fc1)
    objTKSolver.SetValue("Fc2", "i",Fc2)
    objTKSolver.Solve()
    Fo=objTKSolver.GetValue("Fo", "o")
    C=objTKSolver.GetValue("C", "o")
    BW=objTKSolver.GetValue("BW", "o")
    Q=objTKSolver.GetValue("Q", "o")
    
    
    print("\n\n")
    print("OUTPUT")
    
    print("Fo = ",Fo,"Hz")
    print("Fc1 = ",Fc1,"Hz")
    print("Fc2 = ",Fc2,"Hz")
    print("Q = ",Q)
    print("BW = ",BW)
    print("C = ",C)
    
    print("\n\n")
    m=float(1)
    n=float(Fo)*2
    o=float((m+n)/20)
    f_arr=np.arange(m,n,o)
    vg_arr=[]
    gain_in_dB_arr=[]
 
    for i in range(len(f_arr)):
        model = load_fmu ("multifeedbackactivebandpass.fmu")
        model.set("r1.R",R1)
        model.set("r2.R",R2)
        model.set("r3.R",R3)
        model.set("c1.C",C1)
        model.set("c2.C",C2)
        model.set("sineVoltage.V",vin)
        model.set("sineVoltage.freqHz",f_arr[i])

 

        if(f_arr[i]<=float(Fo)):
            res=model.simulate()
            a=max(res['opAmp.out.v'])
            objTKSolver.SetValue("vout", "i",a)
            objTKSolver.Solve()
            voltagegain=objTKSolver.GetValue("voltagegain", "o")
            gain_in_dB=objTKSolver.GetValue("gain_in_dB", "o")
            vg_arr.append(float(voltagegain))
            gain_in_dB_arr.append(float(gain_in_dB))
           
 
    vg_arr=vg_arr+vg_arr[::-1]
    gain_in_dB_arr=gain_in_dB_arr+gain_in_dB_arr[::-1]
    f_arr=f_arr[:len(gain_in_dB_arr)]
    index=1
    for i in f_arr:
        objTKSolver.SetSubCell("l",1,index,1,str(i))
        index=index+1
    
    index=1
    for i in gain_in_dB_arr:
        objTKSolver.SetSubCell("l",2,index,1,str(i))
        index=index+1
    
    P.figure(figsize=(8,6))
    P.grid(True)
    P.plot(f_arr,gain_in_dB_arr,'r',linewidth=2.0,color="magenta")
    P.title("Voltage Gain in dB VS Frequency")
    P.ylabel('Voltage gain in dB')
    P.xlabel('Frequency Hz')
    P.show()
    

 

   
    index1=-1
    index2=-1
    for i in f_arr:  
        if(i<=float(Fc1)):
            index1=index1+1
        else:
            break
    for i in f_arr:
        if(i<=float(Fc2)):
            index2=index2+1
        else:
            break
    
    f_arr=f_arr[index1:index2+1]
    gain_in_dB_arr=gain_in_dB_arr[index1:index2+1]
    P.figure(figsize=(8,6))
    P.grid(True)
    P.plot(f_arr,gain_in_dB_arr,'r',linewidth=2.0,color="orange")
    P.title("Voltage Gain in dB VS Frequency")
    P.ylabel('Voltage gain in dB')
    #P. fill_between(x11, y11, color='lavender')
    P.xlabel('Frequency Hz')
    P.show()
    
    conn = sqlite3.connect('mydb.db')
    c = conn.cursor()
    c.execute(
            "INSERT INTO Multifeedback (R1,R2,R3,C1,C2,vin,Fc1,Fc2,Fo,C,BW,Q) VALUES (?, ?, ?, ?,?, ?, ?, ?,?,?,?,?)",
            (R1,R2,R3,C1,C2,vin,Fc1,Fc2,Fo,C,BW,Q))

    conn.commit()
    conn.close()   
    
    
   
    
                        
                      
 

In [ ]:
def bandstopfilter():
    objTKSolver = win32com.client.Dispatch("TKWX.Document")
    objTKSolver.LoadModel("r", r"C:\Users\HARDIK\UTS_PROJECT\bandstopfilter.tkwx")
    print("\n\n")
    FL=input("Enter value of FL ")
    FH=input("Enter value of FH ")
    R=input("Enter value of Resistor R")
    CL=input("Enter value of Capacitor CL")
    CH=input("Enter value of Capacitor CH")
    vin=input("Enter value of Sine Voltage")
    objTKSolver.SetValue("FL", "i",FL)
    objTKSolver.SetValue("FH", "i",FH)
    objTKSolver.SetValue("CL", "i",CL)
    objTKSolver.SetValue("CH", "i",CH)
    objTKSolver.SetValue("vin", "i",vin)
    objTKSolver.Solve()
    RL=objTKSolver.GetValue("RL", "o")
    RH=objTKSolver.GetValue("RH", "o")
    FC=objTKSolver.GetValue("FC", "o")
    Q=objTKSolver.GetValue("Q", "o")
    FBW=objTKSolver.GetValue("FBW", "o")

    
    print("\n\n")
    print("OUTPUT")
    print("RH = ",RH,"ohms")
    print("RL = ",RL,"ohms")
    print("R = ",R,"ohms")
    print("CL = ",CL,"F")
    print("CH = ",CH,"F")    

    print("FL = ",FL,"Hz")
    print("FH = ",FH,"Hz")
    print("FC = ",FC,"Hz")
    print("Q = ",Q)
    print("FBW = ",FBW)
    
    print("\n\n")
    m=float(1)
    n=float(FC)*2
    o=float((m+n)/20)
    f_arr=np.arange(m,n+(4*o),o) 
    vg_arr=[]
    gain_in_dB_arr=[]

   
    for i in range(len(f_arr)):
        model = load_fmu ("bandstopfilter.fmu")
        model.set("RL.R",RL)
        model.set("RH.R",RH)
        model.set("R1.R",R)
        model.set("R2.R",R)
        model.set("R3.R",R)
        model.set("CL.C",CL)
        model.set("CH.C",CH)
        model.set("sineVoltage.V",vin)
        model.set("sineVoltage.freqHz",f_arr[i])
        if(f_arr[i]<=float(FC)): #200 250 300 350 400 450 500 550 600 650 700 750 800
            res=model.simulate()
            a=max(res['voltageSensor.v'])
            objTKSolver.SetValue("vout", "i",a)
            objTKSolver.Solve()
            voltagegain=objTKSolver.GetValue("voltagegain", "o")
            gain_in_dB=objTKSolver.GetValue("gain_in_dB", "o")
            vg_arr.append(float(voltagegain)) 
            gain_in_dB_arr.append(float(gain_in_dB))
           
              
  
    vg_arr=vg_arr+vg_arr[::-1]
    gain_in_dB_arr=gain_in_dB_arr+gain_in_dB_arr[::-1]
    f_arr=f_arr[:len(gain_in_dB_arr)]
    index=1
    for i in f_arr:
        objTKSolver.SetSubCell("l",1,index,1,str(i))
        index=index+1
    
    index=1
    for i in gain_in_dB_arr:
        objTKSolver.SetSubCell("l",2,index,1,str(i))
        index=index+1
    
    P.figure(figsize=(8,6))
    P.grid(True)
    P.plot(f_arr,gain_in_dB_arr,'r',linewidth=2.0,color="magenta")
    P.title("Voltage Gain in dB VS Frequency")
    P.ylabel('Voltage gain in dB')
    P.xlabel('Frequency Hz')
    P.show()
    

   
    index1=-1
    index2=-1
    for i in f_arr:  
        if(i<=float(FL)):
            index1=index1+1
        else:
            break
    for i in f_arr:
        if(i<=float(FH)):
            index2=index2+1
        else:
            break
    
    f_arr=f_arr[index1:index2+1]
    gain_in_dB_arr=gain_in_dB_arr[index1:index2+1]
    P.figure(figsize=(8,6))
    P.grid(True)
    P.plot(f_arr,gain_in_dB_arr,'r',linewidth=2.0,color="orange")
    P.title("Voltage Gain in dB VS Frequency")
    P.ylabel('Voltage gain in dB')
    #P. fill_between(x11, y11, color='lavender')
    P.xlabel('Frequency Hz')
    P.show()
    conn = sqlite3.connect('mydb.db')
    c = conn.cursor()
    c.execute("INSERT INTO Bandstop (FL,FH,R,CL,CH,vin,RL,RH,FC,Q,FBW) VALUES (?, ?, ?, ?,?, ?, ?, ?,?,?,?)",
            (FL,FH,R,CL,CH,vin,RL,RH,FC,Q,FBW))

    conn.commit()
    conn.close()

    
    

In [ ]:
def rlcseries():
    
    model = load_fmu ("RLCSERIES.fmu")
    objTKSolver = win32com.client.Dispatch("TKWX.Document")
    objTKSolver.LoadModel("r", r"C:\Users\HARDIK\UTS_PROJECT\RLCSERIES.tkwx")
    
    print("\n")
    
    R=input("Enter value of Resistor")
    L=input("Enter value of Inductor")
    C=input("Enter value of Capacitor")
    f=input("Enter value of frequency")
    V=input("Enter value of Sine Voltage")
    print("\n\n")

    objTKSolver.SetValue("R", "i",R)
    objTKSolver.SetValue("L", "i",L)
    objTKSolver.SetValue("C", "i",C)
    objTKSolver.SetValue("f", "i",f)
    objTKSolver.SetValue("Vs", "i",V)

    objTKSolver.Solve()

    XL=objTKSolver.GetValue("XL", "o")
    XC=objTKSolver.GetValue("XC", "o")
    Z=objTKSolver.GetValue("Z", "o")
    I=objTKSolver.GetValue("I", "o")
    Vr=objTKSolver.GetValue("Vr", "o")
    Vl=objTKSolver.GetValue("Vl", "o")
    Vc=objTKSolver.GetValue("Vc", "o")
    phi=objTKSolver.GetValue("Φ", "o")
    Fr=objTKSolver.GetValue("Fr", "o")
   
    print("Inductive Reactance ",XL,"ohms")
    print("Capacitive Reactance ",XC,"ohms")
    print("Circuit Impedance ",Z,"ohms")
    print("Circuits Current ",I,"A")
    print("Voltages across the Resistor ",Vr,"V")
    print("Voltages across the Inductor ",Vl,"V")
    print("Voltages across the Capacitor ",Vc,"V")
    print("Phase Angle ",phi)
    print("Resonant Frequency = ",Fr,"ohms")

    print("\n\n")

    model.set("resistor.R",R)
    model.set("inductor.L",L)
    model.set("capacitor.C",C)
    model.set("sineVoltage.V",V)
    model.set("sineVoltage.freqHz",f)

    res=model.simulate()


    r=model.get("resistor.R")
    l=model.get("inductor.L")
    c=model.get("capacitor.C")
    v=model.get("sineVoltage.V")
    fr=model.get("sineVoltage.freqHz")


    r_res=res['resistor.R']
    l_res=res['inductor.L']
    c_res=res["capacitor.C"]
    v_res=res["sineVoltage.V"]
    f_res=res["sineVoltage.freqHz"]
    i_res=res['currentSensor.i']

    print("\n\n")
    
    fig = P.figure(figsize=(6,6))
    P.subplot(2,1,1)
    P.grid(True)
    P.plot(i_res,'r',linewidth=1.0,color="mediumvioletred")
    P.title("Current ")
    P.ylabel('Current (A)')
    P.xlabel('Time (s)')
    P.show()

    P.subplot(2,1,2)
    P.plot(l_res,linewidth=1.0, marker='o', color="aqua")
    P.title("Inductor ")
    P.grid(True)
    P.ylabel('Inductor (farad)')
    P.xlabel('Time (s)')
    P.show()
    
    f1=1
    f2=float(Fr)*2
    arr=np.arange(f1,f2,1.0)
    F=[]
    F1=[]
    F2=[]
    for i in arr:
        objTKSolver.SetValue("f","i",str(i))
        objTKSolver.Solve()
        a=objTKSolver.GetValue("XL", "o")
        b=objTKSolver.GetValue("XC", "o")
        c=objTKSolver.GetValue("Z", "o")
        F.append(float(a))
        F1.append(float(b))
        F2.append(float(c))
    P.figure(figsize=(8,6))
    P.grid(True)
    P.plot(arr,F,linewidth=1.0,color="lightgreen",marker='o',label='XL')
    P.plot(arr,F1,linewidth=1.0,color="orange",marker='o',label='XC')
    P.plot(arr,F2,linewidth=1.0,color="magenta",marker='o',label='Z')
    P.title("Series Resonance Frequency")
    P.ylabel('Reactance in ohms')
    P.xlabel('frequency (Hz)')
    P.legend()
    P.yscale('log')   
    P.show() 
    conn = sqlite3.connect('mydb.db')
    c = conn.cursor()
    c.execute("INSERT INTO series (R,L,C,f,V,XL,XC,Z,I,Vr,Vl,Vc,phi,Fr) VALUES (?, ?, ?, ?,?, ?, ?, ?,?,?,?,?,?,?)",
            (R,L,C,f,V,XL,XC,Z,I,Vr,Vl,Vc,phi,Fr))

    conn.commit()
    conn.close()
    
    


In [ ]:
def rlcparallel():
    model=load_fmu("RLCPARALLEL.fmu")
    objTKSolver = win32com.client.Dispatch("TKWX.Document")
    objTKSolver.LoadModel("r",r"C:\Users\HARDIK\UTS_PROJECT\RLCPARALLEL.tkwx")
    R=input("Enter value of Resistor")
    L=input("Enter value of Inductor")
    C=input("Enter value of Capacitor")
    f=input("Enter value of frequency")
    V=input("Enter value of Sine Voltage")
    print("\n\n")
    
    objTKSolver.SetValue("R", "i",R)
    objTKSolver.SetValue("L", "i",L)
    objTKSolver.SetValue("C", "i",C)
    objTKSolver.SetValue("f", "i",f)
    objTKSolver.SetValue("V", "i",V)
    
    objTKSolver.Solve()
    
    XL=objTKSolver.GetValue("XL", "o")
    XC=objTKSolver.GetValue("XC", "o")
    Z=objTKSolver.GetValue("Z", "o")
    IR=objTKSolver.GetValue("IR", "o")
    IL=objTKSolver.GetValue("IL", "o")
    IC=objTKSolver.GetValue("IC", "o")
    IS=objTKSolver.GetValue("IS", "o")
    G=objTKSolver.GetValue("G", "o")
    BC=objTKSolver.GetValue("BC", "o")
    Y=objTKSolver.GetValue("Y", "o")
    phi=objTKSolver.GetValue("phi", "o")
    Fr=objTKSolver.GetValue("Fr", "o")
    print("\n\n")
    print("Inductive Reactance ",XL,"ohms")
    print("Capacitive Reactance ",XC,"ohms")
    print("Circuit Impedance ",Z,"ohms")
    print("Current through resistor ",IR,"A")
    print("Current through inductor",IL,"A")
    print("Current through the capacitor ",IC,"A")
    print("Total supply current ",IS,"A")
    print("Conductance",G)
    print("Capacitive  Susceptance",BC)
    print("Admittance",Y)    
    print("Phase Angle ",phi)
    print("Resonant Frequency = ",Fr,"ohms")
    print("\n\n")
    
    print("\n\n")

    model.set("resistor.R",R)
    model.set("inductor.L",L)
    model.set("capacitor.C",C)
    model.set("sineVoltage.V",V)
    model.set("sineVoltage.freqHz",f)


    res=model.simulate()


    r=model.get("resistor.R")
    l=model.get("inductor.L")
    c=model.get("capacitor.C")
    v=model.get("sineVoltage.V")
    fr=model.get("sineVoltage.freqHz")


    r_res=res['resistor.R']
    l_res=res['inductor.L']
    c_res=res["capacitor.C"]
    v_res=res["sineVoltage.V"]
    f_res=res["sineVoltage.freqHz"]
    i_res=res['currentSensor.i']

    print("\n\n")

   
    fig = P.figure(figsize=(6,6))
    P.subplot(2,1,1)
    P.plot(i_res,'r',linewidth=1.0,color="mediumvioletred")
    P.title("Current ")
    P.grid(True)
    P.ylabel('Current (A)')
    P.xlabel('Time (s)')
    P.show()

    P.subplot(2,1,2)
    P.plot(l_res,linewidth=1.0, marker='*', color="aqua")
    P.title("Inductor ") 
    P.grid(True)
    P.ylabel('Indutor (farad)')
    P.xlabel('Time (s)')
    P.show()
    
    f1=1
    f2=float(Fr)*2
    arr=np.arange(f1,f2,1.0)
    F=[]
    F1=[]
    F2=[]
    for i in arr:
        objTKSolver.SetValue("f","i",str(i))
        objTKSolver.Solve()
        a=objTKSolver.GetValue("XL", "o")
        b=objTKSolver.GetValue("XC", "o")
        c=objTKSolver.GetValue("Z", "o")
        F.append(float(a))
        F1.append(float(b))
        F2.append(float(c))
    P.figure(figsize=(8,6))
    P.grid(True)
    P.plot(arr,F,linewidth=1.0,color="lightgreen",marker='o',label='XL')
    P.plot(arr,F1,linewidth=1.0,color="orange",marker='o',label='XC')
    P.plot(arr,F2,linewidth=1.0,color="magenta",marker='o',label='Z')
    P.title("Resonance Frequency")
    P.ylabel('Reactance in ohms')
    P.xlabel('Frequency (Hz)')
    P.legend()
    P.yscale('log')
    P.show() 
    conn = sqlite3.connect('mydb.db')
    c = conn.cursor()
    c.execute("INSERT INTO parallel (R,L,C,f,V,XL,XC,Z,IR,IL,IC,G,BC,Y,phi,Fr) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",(R,L,C,f,V,XL,XC,Z,IR,IL,IC,G,BC,Y,phi,Fr))

    conn.commit()
    conn.close()
    

In [ ]:
def read_data(choice):
    conn=sqlite3.connect('mydb.db')       
    if(choice==1):
        print(pd.read_sql_query("SELECT * FROM Activelowpass1",conn))
    elif(choice==2):
        print(pd.read_sql_query("SELECT * FROM  Activelowpass2",conn))
    elif(choice==3):
        print(pd.read_sql_query("SELECT * FROM  Activelowpass",conn))
    elif(choice==4):
        print(pd.read_sql_query("SELECT * FROM  Activehighpass1",conn))
    elif(choice==5):
        print(pd.read_sql_query("SELECT * FROM Activehighpass2",conn))
    elif(choice==6):
        print(pd.read_sql_query("SELECT * FROM Activehighpass",conn))
    elif(choice==7):
        print(pd.read_sql_query("SELECT * FROM Multifeedback",conn))
    elif(choice==8):
        print(pd.read_sql_query("SELECT * FROM Bandstop",conn))
    elif(choice==9):
        print(pd.read_sql_query("SELECT * FROM series",conn))
    elif(choice==10): 
        print(pd.read_sql_query("SELECT * FROM parallel",conn))
    conn.commit()

In [ ]:
print("                 ACTIVE FILTERS AND RLC CIRCUITS  \n\n")
print("1.ACTIVE LOW PASS FILTER (FIRST ORDER)\n2.ACTIVE LOW PASS FILTER (SECOND ORDER)\n3.ACTIVE LOW PASS FIRST AND SECOND ORDER COMBINED\n4.ACTIVE HIGH PASS FILTER (FIRST ORDER)\n5.ACTIVE HIGH PASS FILTER (SECOND ORDER)\n6.ACTIVE HIGH PASS FIRST AND SECOND ORDER COMBINED\n7.MULTIPLE FEEDBACK FILTER\n8.BAND STOP FILTER\n9.RLC SERIES\n10.RLC PARALLEL\n\n")
choice=int(input("ENTER YOUR CHOICE : "))
if(choice==1):
    lowfirstorder()
elif(choice==2):
    lowsecondorder()
elif(choice==3):
    lowpass_FirstAndSecondOrder()
elif(choice==4):
    highfirstorder()
elif(choice==5):
    highsecondorder()
elif(choice==6):
    highpass_FirstAndSecondOrder()    
elif(choice==7):
    multiplefeedback()
elif(choice==8):
    bandstopfilter()
elif(choice==9):
    rlcseries()
elif(choice==10): 
    rlcparallel()
else:
    print("INVALID CHOICE!! ")

In [ ]:
print("                 DATABASE  \n\n")
print("1.ACTIVE LOW PASS FILTER (FIRST ORDER)\n2.ACTIVE LOW PASS FILTER (SECOND ORDER)\n3.ACTIVE LOW PASS FIRST AND SECOND ORDER COMBINED\n4.ACTIVE HIGH PASS FILTER (FIRST ORDER)\n5.ACTIVE HIGH PASS FILTER (SECOND ORDER)\n6.ACTIVE HIGH PASS FIRST AND SECOND ORDER COMBINED\n7.MULTIPLE FEEDBACK FILTER\n8.BAND STOP FILTER\n9.RLC SERIES\n10.RLC PARALLEL\n\n")
choice=int(input("ENTER YOUR CHOICE : "))
if(choice==1):
    read_data(1)
elif(choice==2):
    read_data(2)
elif(choice==3):
    read_data(3)
elif(choice==4):
    read_data(4)
elif(choice==5):
    read_data(5)
elif(choice==6):
    read_data(6)    
elif(choice==7):
    read_data(7)
elif(choice==8):
    read_data(8)
elif(choice==9):
    read_data(9)
elif(choice==10): 
    read_data(10)
else:
    print("INVALID CHOICE!! ")